In [2]:
import leafmap

ModuleNotFoundError: No module named 'leafmap'

In [1]:
from src import cropy as cpy
import json
import geopandas as gpd
import numpy as np
import rioxarray
import xarray as xr
#https://nbviewer.jupyter.org/github/pangeo-data/cog-best-practices/blob/main/0-single-cog.ipynb
import os
os.environ['GDAL_DISABLE_READDIR_ON_OPEN']='EMPTY_DIR' #This is KEY! otherwise we send a bunch of HTTP GET requests to test for common sidecar metadata
os.environ['AWS_NO_SIGN_REQUEST']='YES' #Since this is a public bucket, we don't need authentication
os.environ['GDAL_MAX_RAW_BLOCK_CACHE_SIZE']='8000000000'  #800MB: Want this to be greater than size of uncompressed raster to overcome a 10 MB limit in the GeoTIFF driver for range request merging.
os.environ['GDAL_SWATH_SIZE']='8000000000'  #also increase this if increasing MAX_RAW_BLOCK_CACHE_SIZE
os.environ['VSI_CURL_CACHE_SIZE']='8000000000' #also increase this if increasing MAX_RAW_BLOCK_CACHE_SIZE8

In [2]:
#dogu ve gap
target_aoi={
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              26.54571533203125,
              37.622933594900864
            ],
            [
              28.56719970703125,
              37.622933594900864
            ],
            [
              28.56719970703125,
              38.85682013474361
            ],
            [
              26.54571533203125,
              38.85682013474361
            ],
            [
              26.54571533203125,
              37.622933594900864
            ]
          ]
        ]
      }
    }
  ]
}

In [3]:
datajson=json.dumps(target_aoi)
area=gpd.read_file(datajson)

In [4]:
#target_aoi=open("target_aoi/tr-west.json")
#area=gpd.read_file(target_aoi)

In [5]:
cpy.VectorProcessing.show_vector(target_area=area)

In [6]:
m,intersec_df=cpy.VectorProcessing.show_intersection(target_area=area,base_vector_path='sentinel_tiles/sentinel_tr_tiles.shp')

In [7]:
m

In [8]:
boundry=list(area.geometry.bounds.values[0]) #boundry from your AOI
bbox=[boundry[0],boundry[1],boundry[2],boundry[3]] #(min lon, min lat, max lon, max lat)
dates = '2021-03-01/2021-05-15'
band_list=['B02','B08']
cloud_percentage=30

In [9]:
stac_query=cpy.Stac(target_aoi=bbox,date=dates,max_cloud=cloud_percentage)

In [10]:
stac_query.find_time_series()

asagida yaptigimiz islem tamamen manuel
once her bir tile icin en buyuk coverage lari alip bir tile listesi yaptim
daha sonra bu tile listesi icinden gorsel olarak iyi oldugunu dusunduklerimin ID kayit altina aldim
sonra bu listeye gore indirme islemini baslattik

In [11]:
tile_list=stac_query.tiles_list

In [12]:
import satstac
def get_stac_tile_item_coverage(stac_items,tile):
    tile_number=int(tile[0:2])
    lat_band=tile[2]
    grid_sq=tile[3:]
    tmp_items=satstac.ItemCollection(stac_items)
    tmp_items.filter('sentinel:utm_zone',[tile_number,])
    tmp_items.filter('sentinel:latitude_band',[lat_band])
    tmp_items.filter('sentinel:grid_square', [grid_sq])
    #try sort
    #return sorted(tmp_items)
    return tmp_items

In [13]:
temp=[get_stac_tile_item_coverage(stac_query.stac_items,tile) for tile in tile_list]

In [14]:
len(temp)

9

## selection
temp olan her bir tile icin zaman araligindakac tane goruntu varsa hepsini overview ustunden bakiyoruz.

In [18]:
cpy.Stac.show_result_map(items_list=temp[4],
                         overview=True,
                         #target_area=target_aoi
                         target_area=area)

In [19]:
#2021 tr bati
input_img_list=['S2A_MSIL2A_20210412T085551_N0300_R007_T35SNC_20210412T121608']
len(input_img_list)
#S2A_MSIL2A_20200605T110631_N0214_R137_T29TQF_20200605T140337

1

### output_img_list

bu liste stac item lardan olusuyor. asagida yapmis oldugumuz sorgu ile goz ile belirledigimiz goruntulerin ID lerini, stac item ID leri ile eslestirerek kendimize
son bir liste olusturuyoruz.

In [21]:
output_img_list=[]
for img in stac_query.stac_items :
    if img.properties['sentinel:product_id'] in input_img_list:
        #print(img)
        output_img_list.append(img)
print(len(output_img_list))

1


In [23]:
output_img_list[0].

S2A_35SNC_20210412_0_L2A

In [24]:
cpy.Stac.show_result_map(items_list=output_img_list,
                         overview=True,
                         #target_area=target_aoi
                         target_area=area)

In [213]:
def download_best_s2(
    item_list,
    band_list,
    cloud_masking,
    download_path,
    send_s3,
    bucket_name):
    cpy.Stac.download_image(item_list=item_list,
                        band_list=band_list,                                           
                        download_path=download_path,
                        cloud_masking=cloud_masking, 
                        send_s3=send_s3,
                        bucket_name=bucket_name
                       )

In [214]:
%%time
download_best_s2(
    item_list=output_img_list,
    band_list=['B02','B03','B04','B08'],
    # after copy to s3, all S2 files will be deleted from download_path
    download_path="./tr-east-gap-20220303",
    cloud_masking=True,
    send_s3=True,
    bucket_name="sentinel2-doktar")

CPU times: user 14min 9s, sys: 8min 45s, total: 22min 55s
Wall time: 1h 1min 12s


In [1]:
import os

In [4]:
os.path.basename('cropy/us-south-20220210')

'us-south-20220210'